In [1]:
import scipy.io
import numpy as np

In [2]:
design_matrix = scipy.io.loadmat('Orange_02_2021_design_matrix_only.mat')['Orange_02_2021_design_matrix']

In [3]:
design_matrix.shape

(2860, 10)

In [4]:
def time_shift_matrix(time_series, before, after):
    padded_time_series = np.pad(time_series, (before, after))
    time_shifts = []
    for i in range(before+after):
        time_shifts.append(padded_time_series[i:-(before+after)+i])
    return time_shifts

In [5]:
before = 3
after = 3
time_indices = [1, 2, 3]

full_design_matrix = []
for i in range(design_matrix.shape[-1] -1):
    if i in time_indices:
        full_design_matrix.extend(time_shift_matrix(design_matrix[:, i], before, after))
    else:
        full_design_matrix.append(design_matrix[:, i])
full_design_matrix = np.stack(full_design_matrix)

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

alpha = 1.0
ridge = Ridge(alpha)
scores = cross_val_score(ridge, full_design_matrix, design_matrix[:, -1], cv=5)
print("%0.2f r^2 with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

ValueError: Found input variables with inconsistent numbers of samples: [24, 2860]

In [25]:
full_design_matrix.shape
#print(full_design_matrix[24,:])
np.any(full_design_matrix<0)

False

As is, full_design_matrix isnt the correct size for what my understanding of what padding/shifting should be yielding. There are 9 behavioral events as covariates we want to include, 3 pre + 3 post + 1 original would be 7 columns for each behavior x 9 behaviors = 63 input columns (also it looks like the dimensions from the original matrix load in have been inverted to create the final full_design_matrix, but for model input each feature should be a column (in line with origianl read in of design matrix))?

Since I can barely maneuver in python lol, I just made the shifted input design matrix, and separate Zdff dependent variable vector in Matlab (same Dropbox folder as the original you already loaded), and am using those here. 

In [43]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

design_matrix = scipy.io.loadmat('Time_shifted_matrices_for_python.mat')['time_shifted_input_design_matrix']
target_Zdff_values=scipy.io.loadmat('Time_shifted_matrices_for_python.mat')['Zdff_values']
alpha = 1
ridge = Ridge(alpha)
ridge.fit(design_matrix,target_Zdff_values)
ridge.get_params(deep=True)
ridge.predict(design_matrix)
ridge.score(design_matrix,target_Zdff_values,sample_weight=None)
#ridge.set_params(alpha=alpha,....) 

0.04812478942553333